In [ ]:
import requests
import pickle
import geopandas as gpd
from pathlib import Path

Set Endpoint for ONS Open Geography Portal (OGP) API

In [ ]:
ENDPOINT = "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Lower_layer_Super_Output_Areas_December_2021_Boundaries_EW_BSC_V4/FeatureServer/0/query"

Function for making requests to OGP and returing results as a GeoDataframe

In [ ]:
def request_to_gdf(url: str, query_params: dict) -> gpd.GeoDataFrame:
    query_params["f"] = "geoJSON"
    response = requests.get(url, query_params)
    if response.ok:
        content = response.json()
        return (
            response,
            gpd.GeoDataFrame.from_features(
                content["features"],
                crs=content["crs"]["properties"]["name"]
            )
        )
    else:
        raise requests.RequestException(
            f"HTTP Code: {response.status_code}, Status: {response.reason}"
        )

Query params for getting all LSOA data  
Requests only the 'code' (LSOA21CD) and 'name' (LSOA21NM) columns for LSOA 

In [ ]:
params = {
    "where": "1=1",
    "outSR": 4326,
    "f": "geoJSON",
    "resultOffset": 0,
    "outFields": "LSOA21CD, LSOA21NM"
}

Results are paginated, so use a loop to grab more pages and concatenate into one big dataframe

In [ ]:
more_pages = True
offset = 0

resp, gdf = request_to_gdf(ENDPOINT, params)

offset = len(gdf)

all_lsoas = gdf
while more_pages:
    try:
        params["resultOffset"] += offset
        response, gdf = request_to_gdf(ENDPOINT, params)
        content = response.json()
        all_lsoas = gpd.pd.concat([all_lsoas, gdf])
        more_pages = content["properties"]["exceededTransferLimit"]
        offset = len(gdf)

    except KeyError:
        more_pages = False

all_lsoas = all_lsoas.reset_index(drop=True)

There should be 35672 LSOAs and 3 columns (geometry, LSOA21CD, LSOA21NM)

In [ ]:
print(f"Rows: {all_lsoas.shape[0]}")
print(f"Column headings: {all_lsoas.columns}")

Make a data directory (if ti doesn't already exist) and save the data as a pkl.

In [ ]:
all_lsoas_file = Path("../data/all_lsoas.pkl").resolve()
all_lsoas_file.parent.mkdir(exist_ok=True)

with open(all_lsoas_file, "wb") as lsoas_file:
    pickle.dump(all_lsoas, lsoas_file)